In [160]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [ ]:
# start = 1719503720
# end = 1719506560


importlib.reload(overview)
anubis_file = "proAnubis_240627_1720.raw"
data, times, fReader = overview.get_chunks(anubis_file, start="2024-06-27 16:55:20", end = "2024-06-27 17:55:20", tdc5 = True)

In [ ]:
storage = "final.pkl"
with open(storage, "wb") as f:
    pickle.dump(data, f)
print("Saved")


In [11]:
importlib.reload(AtTools)
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data("tdc_only.pkl", trigger_channel=4, bcr_channel=2, amount_of_events=784_480 )

In [5]:
data = analyser.anubis_data.copy()

In [12]:
analyser.anubis_data = data.copy()

In [ ]:
analyser.correctionBCR()

In [13]:
importlib.reload(AtTools)
import pandas as pd
only_in = False
if only_in:
    analyser.atlas_data = pd.read_csv("C://Users//jony//Programming//Python//Anubis//anubis//only_in.csv")
else:
    atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//dataAtlas.root"
    analyser.getAtlasData(atlas_file)
    #analyser.atlas_data = analyser.atlas_data.iloc[len(analyser.atlas_data)//2:]
print("Loading done")

Loading done


In [4]:
with tqdm(total=len(analyser.atlas_data.values)) as pbar:
    good_indices = []
    for i in range(len(analyser.atlas_data.values)):
        hits = analyser.atlas_data.iloc[i]
        for p in range(len(hits["mu_eta"])): #0.93, 0.83, 1.57
            if 1.0 > hits["mu_eta"][p] > 0.70 and abs(hits["mu_phi"][p] - 1.57) < 0.4:
                good_indices.append(i)
                break
        pbar.update(1)
analyser.atlas_data = analyser.atlas_data.iloc[good_indices]
print(analyser.atlas_data)

KeyboardInterrupt: 

In [ ]:
muons = 0

for i in range(len(analyser.atlas_data.values)):
    hits = analyser.atlas_data.iloc[i]
    for p in range(len(hits["mu_eta"])):
        if 0.93 > hits["mu_eta"][p] > 0.83 and abs(hits["mu_phi"][p] - 1.57) < 0.1:
            muons += 1
            print(hits["mu_eta"][p], hits["mu_phi"][p])

print(muons)
print(len(analyser.atlas_data.values)/(analyser.atlas_data.iloc[-1]["TimeStamp"]-analyser.atlas_data.iloc[0]["TimeStamp"]))

In [ ]:
#plot density by eta
hist_eta = []
for i in range(len(analyser.atlas_data.values)):
    hits = analyser.atlas_data.iloc[i]
    for p in range(len(hits["mu_eta"])):
            hist_eta.append(hits["mu_eta"][p])
plt.title("Density of muons by eta")
plt.xlabel("Eta")
plt.xlim(-3, 3)
plt.hist(hist_eta, bins=100, density=True)
plt.show()


In [14]:
# set the times
analyser.atlas_data = analyser.atlas_data[analyser.atlas_data["TimeStamp"] > analyser.anubis_data[0].timeStamp]
print("Anubis", (round(analyser.anubis_data[0].timeStamp), round(analyser.anubis_data[-1].timeStamp)))
print("ATLAS",(analyser.atlas_data.iloc[0]["TimeStamp"], analyser.atlas_data.iloc[-1]["TimeStamp"]))

Anubis (1719503720, 1719506762)
ATLAS (np.uint32(1719503721), np.uint32(1719506684))


In [98]:
times = analyser.atlas_data["TimeStamp"]
print(times) 
#1719504220
#1719503530
hist, bins = np.histogram(times, bins=100)
plt.plot(bins[:-1], hist)
plt.show()

9378      1719503721
9380      1719503721
10762     1719503721
10919     1719503721
9367      1719503721
             ...    
144140    1719506684
145102    1719506684
145264    1719506684
144324    1719506684
144329    1719506684
Name: TimeStamp, Length: 135594, dtype: uint32


In [ ]:
test = analyser.atlas_data[analyser.atlas_data["LumiBlock"] == 516].copy()
print(test[test["BCID"] ==  2799])

In [152]:
#Calculate the BCR offset
importlib.reload(AtTools)
anubis = AtTools.BCRHistogram(analyser.anubis_data, atlas=False, plot = False)
atlas = AtTools.BCRHistogram(analyser.atlas_data, atlas=True, plot = False)

window_size = 100
anubis = np.convolve(np.append(anubis, anubis[:window_size]), np.ones(window_size)/window_size, mode='valid')
atlas = np.convolve(np.append(atlas, atlas[:window_size]), np.ones(window_size)/window_size, mode='valid')
error = 100
errors = []

"""
for offset in range(-200,200):
    errors.append(np.sum((atlas - np.roll(anubis, offset))**2))
    if np.sum((atlas - np.roll(anubis, offset))**2) < error:
        error = np.sum((atlas - np.roll(anubis, offset))**2)
        best_offset = offset
"""
best_offset = 114
print("Best offset", best_offset)
plt.plot(atlas, label="Atlas")
#plt.plot(anubis, label="Original Anubis")np.roll(anubis, best_offset)
plt.plot(anubis, label="Shifted proAnubis") 
plt.xlim(0, 3565)
plt.ylabel("Counts (normalized)")
plt.xlabel("BX")
plt.title("proAnubis and Atlas Bunch Crossing Histograms - Aligned")
plt.legend()
plt.show()

plt.plot(list(range(-200,200)), errors, label="Errors")
plt.xlabel("Offset")
plt.ylabel("Sum of Square Errors")
plt.title("Sum of Square Errors vs Offset")
plt.vlines(best_offset,0, max(errors), color = "red", label=f"Best Offset: {best_offset}")
plt.ylim(0,max(errors))
plt.legend(loc = "lower left") #position

plt.show()
    

Best offset 114


ValueError: x and y must have same first dimension, but have shapes (400,) and (0,)

In [110]:
importlib.reload(AtTools)
AtTools.BCRHistogram(analyser.anubis_data)
#AtTools.BCRHistogram(analyser.atlas_data, atlas=True)


array([2.14302321e-04, 3.01949196e-04, 3.84209950e-04, ...,
       1.18331442e-05, 1.15719989e-04, 1.44527575e-04])

In [15]:
importlib.reload(AtTools) #5111
analyser.pairBCRwithEvents()

Matching: 100%|██████████| 135594/135594 [03:28<00:00, 650.79Events/s]


[[{'RunNumber': 479103,
   'EventNumber': 0,
   'LumiBlock': 511,
   'TimeStamp': 1719503721,
   'TimeStampNS': 64055,
   'BCID': 1093,
   'extL1ID': 1929751570,
   'triggerType': 140,
   'mu_pt': <Array [17.8, 7.4] type='2 * float32'>,
   'mu_eta': <Array [-1.63, 0.581] type='2 * float32'>,
   'mu_phi': <Array [2.2, 2.61] type='2 * float32'>,
   'mu_E': <Array [47.3, 8.69] type='2 * float32'>,
   'mu_charge': <Array [-1, -1] type='2 * int32'>},
  []],
 [{'RunNumber': 479103,
   'EventNumber': 0,
   'LumiBlock': 511,
   'TimeStamp': 1719503721,
   'TimeStampNS': 19257435,
   'BCID': 520,
   'extL1ID': 1929753336,
   'triggerType': 140,
   'mu_pt': <Array [87.3, 4.11, 6.51, 3.79, 2.43] type='5 * float32'>,
   'mu_eta': <Array [0.837, 2.43, 0.794, -1.39, 1.67] type='5 * float32'>,
   'mu_phi': <Array [2.11, -1.33, -0.982, 3.02, -0.255] type='5 * float32'>,
   'mu_E': <Array [120, 23.5, 8.68, 8.06, 6.66] type='5 * float32'>,
   'mu_charge': <Array [-1, 1, 1, -1, -1] type='5 * int32'>},
  

In [112]:
print(len(analyser.atlas_data))

135594


In [157]:
#difference
best_offset = 114
eta = []
phi = []
times = []
matches = [x for x in analyser.matches if x[1]]
for i in range(len(matches)):
    atlas_hits = matches[i][0]
    mu_eta = list(atlas_hits["mu_eta"])
    mu_phi = list(atlas_hits["mu_phi"])

    for p in range(len(mu_phi)): 
        #if 0.95 > mu_eta[p] > 0.8 and abs(mu_phi[p]) < 0.1:        
        if matches[i][1]:
            eta.extend(mu_eta)
            phi.extend(mu_phi)        
            for trigger in matches[i][1]:
                if round(trigger.bcId+best_offset-matches[i][0]["BCID"]) < 3500:
                #if abs(round(trigger.bcId-matches[i][0]["BCID"])) > 1000:
                #    continue
                    times.append(round(trigger.bcId+best_offset-matches[i][0]["BCID"]))
            break

In [95]:
# Define the step sizes for x and y
phi_step = 0.1  # Step size for x-axis
eta_step = 0.2  # Step size for y-axis

# Generate the ranges based on step size
eta_range = np.arange(-2.5, 2.5 + eta_step, eta_step)  # Range from -2.5 to 2.5
phi_range = np.arange(-np.pi, np.pi + phi_step, phi_step)  # Range from -pi to pi

# Generate the grid using meshgrid
phi_coordinates, eta_coordinates = np.meshgrid(phi_range, eta_range)
#print(phi_coordinates) #changes phi
#print(eta_coordinates) #changes eta

# Iterate over every single point in the grid
buffer = matches.copy()
objects = [[[] for i in range(phi_coordinates.shape[1])] for j in range(phi_coordinates.shape[0])]
with tqdm(total=len(buffer)) as pbar:
    for hit in matches:
        found = False
        pbar.update(1)
        for i in range(phi_coordinates.shape[0]):
            for j in range(phi_coordinates.shape[1]):
                phi_val = phi_coordinates[i, j]
                eta_val = eta_coordinates[i, j]
                #print(f"Point ({phi_val}, {eta_val})")
                for p in range(len(hit[0]["mu_eta"])):
                    if eta_val+eta_step/2 > hit[0]["mu_eta"][p] > eta_val-eta_step/2 and abs(hit[0]["mu_phi"][p] - phi_val) < phi_step/2:
                        for trigger in hit[1]:
                            if abs(round(trigger.bcId+best_offset-matches[i][0]["BCID"])) < 1000:
                                objects[i][j].append(round(trigger.bcId+best_offset-matches[i][0]["BCID"])) #add times
                        found = True
                        break
                if found:
                    break
            if found:
                break            

100%|██████████| 6330/6330 [01:49<00:00, 57.96it/s]


In [96]:
from collections import Counter
std_grid = 1000*np.ones((phi_coordinates.shape[0], phi_coordinates.shape[1]))

minimum = 10_000
# Iterate through the grid cells
for i in range(len(objects)):
    for j in range(len(objects[i])):
        if len(objects[i][j]) > 0:
            # Calculate the standard deviation of the numbers in this grid cell
            std_grid[i][j] = np.std(objects[i][j])#(Counter(objects[i][j]).most_common(1)[0][1]) if len(objects[i][j]) > 1 else 0
            if minimum > std_grid[i][j]:
                minimum = std_grid[i][j]
                print((i,j))
                print(phi_coordinates[i, j], eta_coordinates[i, j])
# Output the standard deviation array
print("Standard deviation for each grid cell:")
print(std_grid)


(0, 0)
-3.141592653589793 -2.5
(0, 2)
-2.941592653589793 -2.5
(0, 27)
-0.4415926535897907 -2.5
(0, 35)
0.35840734641021 -2.5
(0, 37)
0.5584073464102102 -2.5
(0, 38)
0.6584073464102103 -2.5
(0, 40)
0.8584073464102104 -2.5
(1, 0)
-3.141592653589793 -2.3
(1, 1)
-3.041592653589793 -2.3
(1, 11)
-2.041592653589792 -2.3
(3, 0)
-3.141592653589793 -1.8999999999999995
(4, 6)
-2.5415926535897926 -1.6999999999999993
(4, 18)
-1.3415926535897915 -1.6999999999999993
(4, 29)
-0.24159265358979054 -1.6999999999999993
(4, 37)
0.5584073464102102 -1.6999999999999993
(4, 44)
1.2584073464102108 -1.6999999999999993
(6, 23)
-0.8415926535897911 -1.299999999999999
(10, 0)
-3.141592653589793 -0.4999999999999982
(10, 9)
-2.2415926535897923 -0.4999999999999982
(13, 44)
1.2584073464102108 0.10000000000000231
Standard deviation for each grid cell:
[[ 604.64064587  606.58185612  555.71796228 ...  594.73603606
   534.92553748  587.84871881]
 [ 397.79639946  369.2713917   401.08343131 ...  381.34553965
   384.28663535  

In [158]:
hits = []
for i in range(len(matches)):
    for trigger in matches[i][1]:
        hits.append(trigger.bcId + best_offset - matches[i][0]["BCID"])

#strange number of problems ~ 8193
bins = [i for i in range(0,3565)]
counts, _ = np.histogram(hits, bins=bins, density=True)
    
bins = bins[:-1]
plt.step(bins, counts, color="orange")

plt.xlabel('Time since last BCR (ns)')
plt.xlim(0, 3565)
plt.ylabel('Frequency')
plt.title(f'Histogram of BCID Matches')
plt.show()


In [94]:
#plot std_grid
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(phi_coordinates, eta_coordinates, std_grid, cmap='plasma')
ax.set_xlabel("Phi")
ax.set_ylabel("Eta")
plt.show()

In [125]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming std_grid is already calculated and phi_coordinates, eta_coordinates are defined

# Plotting a heatmap of std_grid
plt.imshow(std_grid, cmap='plasma', extent=[eta_coordinates.min(), eta_coordinates.max(),
                                            phi_coordinates.min(), phi_coordinates.max()],
           origin='lower', aspect='auto')

plt.colorbar(label='Standard Deviation')
plt.xlabel("Eta")
plt.ylabel("Phi")
plt.title("Heatmap of Standard Deviations")
plt.show()


KeyboardInterrupt: 

In [31]:
#3D plot

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(eta, phi, times)
ax.set_xlabel('eta')
plt.ylim(-3.2,3.2)
plt.xlim(-3.2,3.2)
ax.set_ylabel('phi')
ax.set_zlabel('time')
plt.show()

ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (25,) and arg 2 with shape (175,).

In [159]:
#Histograms

plt.hist2d(eta, phi, bins=30, cmap='plasma')
plt.xlim(-np.pi, np.pi)
plt.ylim(-np.pi, np.pi)
plt.xlabel("eta")
plt.ylabel("phi")
# Add a color bar to show the color scale
plt.colorbar()
plt.show()

plt.step(range(min(times), max(times) + 1, 1), np.histogram(times, bins=range(min(times), max(times) + 2, 1))[0], label = "(phi, eta) = (0, 0.83)")
plt.xlim(-100,100)
plt.xlabel("Time difference (bx)")
#plt.xlim(-500,500)
plt.ylabel("Frequency")
plt.title("Bunch Crossing difference")
plt.legend()
plt.show()

In [142]:
print(len(times))

309


In [123]:
plt.hist(times, bins=range(min(times), max(times) + 2, 1))
#plt.xlim(-100,100)
plt.xlabel("Time difference (bx)")
plt.xlim(-500,500)
plt.ylabel("Frequency")
plt.title("Time difference between BCR and trigger")
plt.show()

In [19]:

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

hist, xedges, yedges = np.histogram2d(eta, phi, bins=30, range=[[-np.pi, np.pi], [-np.pi, np.pi]])

# Prepare the grid for the surface plot
xpos, ypos = np.meshgrid(xedges[:-1] + 0.5 * (xedges[1] - xedges[0]),
                         yedges[:-1] + 0.5 * (yedges[1] - yedges[0]),
                         indexing="ij")

# Initialize the 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
ax.plot_surface(xpos, ypos, hist, cmap='plasma')

# Set labels
ax.set_xlabel('eta')
ax.set_ylabel('phi')
ax.set_zlabel('Count')

# Set limits
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)

# Show the plot
plt.show()
